In [6]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors, decoders
from datasets import load_from_disk


In [3]:
jsonl_dataset = load_from_disk('Processed_BCB_code')

In [4]:
jsonl_dataset

Dataset({
    features: ['func', 'idx', 'edges', 'orders', 'values'],
    num_rows: 9126
})

In [5]:
texts = jsonl_dataset["func"]
texts

['public class DummyClass {\npublic static void main(String[] args) {\n        LogFrame.getInstance();\n        for (int i = 0; i < args.length; i++) {\n            String arg = args[i];\n            if (arg.trim().startsWith(DEBUG_PARAMETER_NAME + "=")) {\n                properties.put(DEBUG_PARAMETER_NAME, arg.trim().substring(DEBUG_PARAMETER_NAME.length() + 1).trim());\n                if (properties.getProperty(DEBUG_PARAMETER_NAME).toLowerCase().equals(DEBUG_TRUE)) {\n                    DEBUG = true;\n                }\n            } else if (arg.trim().startsWith(MODE_PARAMETER_NAME + "=")) {\n                properties.put(MODE_PARAMETER_NAME, arg.trim().substring(MODE_PARAMETER_NAME.length() + 1).trim());\n            } else if (arg.trim().startsWith(AUTOCONNECT_PARAMETER_NAME + "=")) {\n                properties.put(AUTOCONNECT_PARAMETER_NAME, arg.trim().substring(AUTOCONNECT_PARAMETER_NAME.length() + 1).trim());\n            } else if (arg.trim().startsWith(SITE_CONFIG_URL

In [7]:
# 1️⃣ Tạo tokenizer với mô hình BPE (Byte-Pair Encoding)
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

# 2️⃣ Dùng pre-tokenizer giống RoBERTa (byte-level và xử lý space)
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)

# 3️⃣ Định nghĩa trainer cho RoBERTa-style tokenizer
trainer = trainers.BpeTrainer(
    vocab_size=50265,  # RoBERTa dùng 50,265 tokens
    min_frequency=2,
    special_tokens=["<s>", "</s>", "<unk>", "<pad>", "<mask>"]
)

# 4️⃣ Train trực tiếp từ dataset["func"]
tokenizer.train_from_iterator(jsonl_dataset["func"], trainer)

# 5️⃣ Định nghĩa post-processor giống RoBERTa
tokenizer.post_processor = processors.TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> </s> $B </s>",
    special_tokens=[("<s>", 0), ("</s>", 1)]
)

# 6️⃣ Dùng decoder giống RoBERTa
tokenizer.decoder = decoders.ByteLevel()

print("✅ Tokenizer RoBERTa đã train xong!")

✅ Tokenizer RoBERTa đã train xong!


In [8]:
print("Vocabulary size thực tế:", tokenizer.get_vocab_size())

Vocabulary size thực tế: 50265


In [9]:
tokenizer.save("java_tokenizer.json")
print("✅ Tokenizer đã được lưu thành công!")

✅ Tokenizer đã được lưu thành công!
